In [56]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.utils import np_utils
import pandas as pd
import numpy as np
import sys
sys.path.append('/localscratch/data/project_one/update/')
import util
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
_input = pd.read_pickle('/localscratch/data/project_one/data/chart/stocks/ALV.DE.p')
_classification = pd.read_pickle('/localscratch/data/project_one/data/classification/stocks/duration_3/ALV.DE.p')

print _input.columns
noFeatures =['Date','Close'] +  [X for X in _input.columns if X[-1] == 'X'] 

common_dates =util.find_common_notnull_dates(_input,_classification)

_input  = _input.loc[_input['Date'].isin(common_dates)]
Xfull  = _input.loc[:,_input.columns.isin(noFeatures) == False].values

_classification = _classification.loc[_classification['Date'].isin(common_dates)]
Yfull = _classification.loc[:,_classification.columns.isin(['Date']) == False].values*1.0



Xfull -= np.mean(Xfull,axis=0)
Xfull /= np.std(Xfull,axis=0)


print Xfull.shape,Yfull.shape
print _input[10:11]

Index([u'Date', u'Close', u'Volume', u'GD200', u'GD200X', u'GD100', u'GD100X',
       u'GD50', u'GD50X', u'GD38', u'GD38X', u'Lower_BB_20_2',
       u'Upper_BB_20_2', u'Middle_BB_20_2', u'RSI7', u'RSI7X', u'RSI14',
       u'RSI14X', u'RSI25', u'RSI25X', u'WR14', u'WR14X', u'CCI20', u'CCI20X',
       u'PDI14R', u'NDI14R', u'PDI14RX', u'NDI14RX', u'MACD', u'MACDX',
       u'MACDH', u'MACDHX', u'MAX20', u'MAX65', u'MAX130', u'MAX260', u'MIN20',
       u'MIN65', u'MIN130', u'MIN260', u'PVOH', u'PVOHX', u'TRIX', u'TRIXH',
       u'TRIXHX', u'RSV14', u'RSVH14', u'RSVH14X'],
      dtype='object')
(4361, 29) (4361, 12)
          Date  Close    Volume     GD200    GD200X     GD100    GD100X  \
243 2001-01-04  388.0  803823.0 -0.008911  0.006155  0.002595  0.016066   

         GD50   GD50X      GD38    ...       MIN130    MIN260     PVOH  \
243 -0.013788  0.0018 -0.018173    ...     0.094654  0.227848  1.31334   

        PVOHX      TRIX     TRIXH   TRIXHX      RSV14    RSVH14   RSVH14X  
243 -

In [93]:
def convert_to_squence(N_seq,X,Y):
    
    X -=np.mean(X)
    X /=np.std(X)
    
    Y_new = np.zeros(len(X)-N_seq)
    Y_new = [Y[N_seq+i] for i in range(len(Y_new))]
    
    X_new = [X[i:(i+N_seq),:] for i in range(len(Y_new))]
    
    return np.array(X_new),np.array(Y_new)
    
Xnew,Ynew = convert_to_squence(10,Xfull,Yfull)


print Ynew.shape

(4351, 12)


In [94]:
Xtrain = Xnew[0:-200,:,:]
Ytrain = Ynew[0:-200,:]

Xtest = Xnew[-200:,:,:]
Ytest = Ynew[-200:,:]

print Xtrain.shape,Xtest.shape

(4151, 10, 29) (200, 10, 29)


In [62]:
S = np_utils.to_categorical(np.argmax(Ytest,axis=1),num_classes=12)

In [67]:
print S[2,:]
print Ytest[2,:]

[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]


In [69]:
print Xtrain.shape[1:]

(10, 29)


In [ ]:
model = Sequential()
model.add(LSTM(100,dropout=0.2,recurrent_dropout=0.2,input_shape=(None,29)))
model.add(Dense(50,activation='sigmoid'))
model.add(Dense(12,activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
#monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
print(model.summary())
model.fit(Xtrain,Ytrain,validation_data=(Xtest,Ytest),verbose=2,epochs=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 100)               52000     
_________________________________________________________________
dense_39 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_40 (Dense)             (None, 12)                612       
Total params: 57,662
Trainable params: 57,662
Non-trainable params: 0
_________________________________________________________________
None
Train on 4151 samples, validate on 200 samples
Epoch 1/200
 - 10s - loss: 2.3564 - acc: 0.1479 - val_loss: 1.9595 - val_acc: 0.2900
Epoch 2/200
 - 3s - loss: 2.2770 - acc: 0.1843 - val_loss: 1.9555 - val_acc: 0.2500
Epoch 3/200
 - 2s - loss: 2.2649 - acc: 0.1879 - val_loss: 1.9135 - val_acc: 0.2600
Epoch 4/200
 - 2s - loss: 2.2548 - acc: 0.1850 - val_loss: 1.8912 - val_acc: 0.2800
Epoch 5/200
 

In [104]:
score,acc = model.evaluate(Xtest,Ytest)

print score
print acc

p_test = model.predict_proba(Xtest)

200/200 [==============================] - 0s 2ms/step
0.262516791821
0.912083296776
200/200 [==============================] - 2s 8ms/step


In [105]:
pred =  np.argmax(p_test,axis=1)
true= np.argmax(Ytest,axis=1)
print np.sum((pred == true) *1.0)/len(true)

0.26
